In [1]:
%%capture
!pip install pytorch_lightning
!pip install transformers

In [1]:
from torch_geometric.nn import GCNConv, GATConv,DenseGCNConv

In [3]:
from torch.utils.data import DataLoader, random_split
# from torchvision.datasets import MNIST
import os
# from torchvision import datasets, transforms
from torch.optim import Adam
from torch import optim as optim
from torch.nn.functional import normalize
from torch.utils.data import DataLoader, random_split
import os
from matplotlib import pyplot as plt
from pytorch_lightning import Trainer
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule
import random
import math
from torch_sparse import SparseTensor
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import AdamW
from torch.nn.parameter import Parameter
from torch_geometric.nn import Sequential, TransformerConv,BatchNorm,Set2Set
from torch.nn.functional import normalize
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn.functional as F
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
import numpy as np
import pytorch_lightning as pl
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'

In [5]:
import numpy as np
fullsamples = np.load('testEpi.npy')
len(fullsamples )
# fullsamples = np.load('testhetero.npy')
# fullsamples

147

In [ ]:
fullsamples=np.load('/workspace/testEpi.npy')
xyz=[]
adj = []
interface_adj = []
interface = []
sample_file = []
edges = []

for i in range(len(fullsamples)):
    edgesrec = []
    edgeslig = []
#     print(fullsamples[i])
    try:
#         try:
        datain= torch.load(os.path.join('/workspace/EpipredFeat/'+fullsamples[i]+'_input_feat.pt'))
#         except:
#             datain= torch.load(os.path.join('/workspace/EpipredFeat/'+str(fullsamples[i]).split('_')[0].lower()+'.'+str(fullsamples[i]).split('_')[1].split(':')[0]+'.'+str(fullsamples[i]).split('_')[1].split(':')[1]+'.'+'_input_feat.pt'))
        pssm = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_pssm.pt'))
        edgesrec = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_rec.pt'))
        edgeslig = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_lig.pt'))
    except:
            continue

    if not(datain.shape[0]==pssm.shape[0] ):
        print('mismatch')
        continue
#     print(datain.shape)
    feat = torch.hstack((datain, pssm))
    rcc=feat[:,46].cpu()
    unique, counts = np.unique(rcc, return_counts=True)
    coun = dict(zip(unique, counts))
    rec_c = coun[0.0]
    x1,x2,x3 = torch.from_numpy(np.random.uniform(3.14/12, 3.14, 3))
    cos1=torch.cos(x1)
    cos2=torch.cos(x2)
    cos3=torch.cos(x3)
    sin1=torch.sin(x1)
    sin2=torch.sin(x2)
    sin3=torch.sin(x3)
    RMatrix=torch.tensor([[cos1*cos2*cos3-sin1*sin3,cos3*cos2*sin1+sin3+cos1,-cos3*sin2 ],\
                            [-sin3*cos2*cos1-cos3*sin1, -sin3*cos2*sin1+cos3*cos1, sin3*sin2],\
                            [sin2*cos1,sin2*sin1,cos2]])
    coordinates = feat[rec_c:, :3]
    ligcoord=  ((RMatrix @ torch.tensor(coordinates).double().T)).T
    receptor = feat[:rec_c, :] #feat[:rec_c, 3:]   torch.hstack((feat[:rec_c, 3:26], feat[:rec_c, 46:]))
    ligand = torch.hstack((ligcoord, feat[rec_c:, 3:])) #feat[rec_c:, 3:]  torch.hstack((feat[rec_c:, 3:26], feat[rec_c:, 46:])) #feat[rec_c:, 3:]

    if receptor.shape[0]>999 or receptor.shape[0]<80 or ligand.shape[0]>999 or ligand.shape[0]<80:
        continue

    xyz.append((receptor,ligand))
    edges.append((edgesrec, edgeslig))
    adj_rec = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_rec.pt')) 
    adj_lig = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_lig.pt')) 
#     adj_reclig = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_rec_ligAdj.pt')) 
    interface_ = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_interface.pt'))
    adj_rec = adj_rec + torch.eye(adj_rec.shape[0])
    adj_lig = adj_lig + torch.eye(adj_lig.shape[0])
    adj.append((adj_rec,adj_lig))
#     interface_adj.append(adj_reclig)
    interface.append(interface_)
    sample_file.append((fullsamples[i]+'1',fullsamples[i]+'2'))
    
# print('lenght of XYZ:',len(xyz))

In [7]:
class ProteinData(pl.LightningDataModule):
    def __init__(self,samplename, features,adjacency, interface, batch_size = 1, train_val_test_split = [0.7,0.1,0.2]): 
        super().__init__()
        self.features = features
        self.adjacency = adjacency
        self.interface = interface
        self.split = train_val_test_split
        self.batch_size = batch_size
#         self.rec_lig_adjacency = rec_lig_adjacency
        self.samplen = samplename
        self.setup()
        self.db5 = np.load('testEpi.npy').tolist()
                
    def setup(self, stage = None):
        self.data= []
        
        for i in range(len(self.features)):
#             print(self.features[i][0].shape)
            self.data.append([i, self.features[i][0],self.features[i][1],self.adjacency[i][0],self.adjacency[i][1],[],self.interface[i]])
        
        
        self.one = int(self.split[0]*len(self.data))
        self.two = int((self.split[0]+self.split[1])*len(self.data))
#         print(self.one, self.two)
        
        
        self.train, self.val, self.test = [],[],self.data
        
    def length(self):
        print(len(self.train), len(self.val), len(self.data))
#         print(self.one, self.two)
        
    
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size,shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)        
#         return DataLoader(self.test, batch_size=self.batch_size)

In [ ]:
dataobj = ProteinData(sample_file,xyz,adj,interface)
dataobj.length()


In [41]:
def degreeMAT(matrix):
    diag = np.zeros_like(matrix)
    for i in range(len(matrix)):
        np.fill_diagonal(diag[i],np.asarray(matrix[i].sum(axis=-1)).flatten())
    return diag

class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(self.in_features,self.out_features))
        self.relu = nn.LeakyReLU()
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.out_features)
        self.weight.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        dMAT=degreeMAT(adj.cpu())
        dMAT=torch.from_numpy(dMAT).to(adj.device)
        dMAT=torch.linalg.inv(torch.sqrt(dMAT))
        output=dMAT@adj@dMAT@input@self.weight
        return self.relu(output)

In [42]:
def attention(q, k, v, d_k, dropout=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    scores = F.softmax(scores, dim=-1)
    if dropout is not None:
        scores_ = dropout(scores)
    output = torch.matmul(scores_.transpose(-2, -1), v)
    return output, scores

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        
        # calculate attention using function we will define next
        scores, att_map = attention(q, k, v, self.d_k, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output, att_map

In [43]:
class NetR(nn.Module):
    def __init__(self):
        super().__init__()
        self.h_feats_dim = 64#  128 #
        self.heads = 4
        self.out_head = 1
        self.fc1 = DenseGCNConv(64,128) # GATConv(64, 50,self.heads)
        self.fc2 = DenseGCNConv(128, 256) # GATConv(50*self.heads, 64, self.heads)
        self.fc3 = DenseGCNConv(256, 512) #GATConv(64*self.heads, 128, self.heads)
        self.fc4 = DenseGCNConv(512,1024) #GATConv(128*self.heads, 256, self.heads)
        self.fc5 = DenseGCNConv(1024, 512) #GATConv(256*self.heads, 64, self.heads)
        self.fc6 = DenseGCNConv(512,256) #GATConv(64*self.heads, 32, self.heads)
        self.fc7 = DenseGCNConv(256,64) #GATConv(32*self.heads, self.h_feats_dim, self.out_head)
        self.relu = nn.LeakyReLU() #nn.GELU() #
        self.dropout= nn.Dropout(p=0.2)
    def forward(self, x, edge_index):
            x1=F.relu(self.fc1(x, edge_index))
            x1 = F.layer_norm(x1,[x1.size()[-1]])
            x2=F.relu(self.fc2(x1, edge_index)) #self.dropout()
            x2 = F.layer_norm(x2,[x2.size()[-1]])
            x3=F.relu(self.fc3(x2, edge_index))
            x3 = F.layer_norm(x3,[x3.size()[-1]])
            x4=F.relu(self.fc4(x3, edge_index))
            x4 = F.layer_norm(x4,[x4.size()[-1]])
            x5=F.relu(self.fc5(x4, edge_index))
            x5 = F.layer_norm(x5,[x5.size()[-1]])
            x6 =F.relu(self.fc6(x5, edge_index))
            x6 = F.layer_norm(x6,[x6.size()[-1]])
            x7 = self.fc7(x6, edge_index)

            return x7
class NetL(nn.Module):
    def __init__(self):
        super().__init__()
        self.h_feats_dim = 64# 128 #
        self.h_feats_dim = 64#  128 #
        self.heads = 4
        self.out_head = 1
        self.fc1 = DenseGCNConv(64,128) # GATConv(64, 50,self.heads)
        self.fc2 = DenseGCNConv(128, 256) # GATConv(50*self.heads, 64, self.heads)
        self.fc3 = DenseGCNConv(256, 512) #GATConv(64*self.heads, 128, self.heads)
        self.fc4 = DenseGCNConv(512,1024) #GATConv(128*self.heads, 256, self.heads)
        self.fc5 = DenseGCNConv(1024, 512) #GATConv(256*self.heads, 64, self.heads)
        self.fc6 = DenseGCNConv(512,256) #GATConv(64*self.heads, 32, self.heads)
        self.fc7 = DenseGCNConv(256,64) #GATConv(32*self.heads, self.h_feats_dim, self.out_head)
        self.relu = nn.LeakyReLU() #nn.GELU() #
        self.dropout= nn.Dropout(p=0.2)
    def forward(self, x, edge_index):
            x1=F.relu(self.fc1(x, edge_index))
            x1 = F.layer_norm(x1,[x1.size()[-1]])
            x2=F.relu(self.fc2(x1, edge_index))
            x2 = F.layer_norm(x2,[x2.size()[-1]])
            x3=F.relu(self.fc3(x2, edge_index))
            x3 = F.layer_norm(x3,[x3.size()[-1]])
            x4=F.relu(self.fc4(x3, edge_index))
            x4 = F.layer_norm(x4,[x4.size()[-1]])
            x5=F.relu(self.fc5(x4, edge_index))
            x5 = F.layer_norm(x5,[x5.size()[-1]])
            x6 =F.relu(self.fc6(x5, edge_index))
            x6 = F.layer_norm(x6,[x6.size()[-1]])
            x7 = self.fc7(x6, edge_index)
            return x7


In [44]:
class NetRSPA(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = GraphConvolution(128, 128)
        self.l2 = GraphConvolution(256, 256)
        self.l3 = GraphConvolution(512, 512)
        self.l4 = GraphConvolution(1024, 1024)
        self.attn1 = MultiHeadAttention(8, 128)
        self.attn2 = MultiHeadAttention(8, 256)
        self.attn3 = MultiHeadAttention(8, 512)
        self.attn4 = MultiHeadAttention(8, 1024)
        self.h1 = nn.Linear(3,128)
        self.h2 = nn.Linear(128,256)
        self.h3 = nn.Linear(256,512)
        self.h4 = nn.Linear(512,1024)
        self.att_conv = nn.Conv2d(16, 1, kernel_size=1, stride=1,padding='same',bias=True)
        self.linear_out = nn.Sequential(nn.Linear(1024,128),nn.LeakyReLU(),nn.Linear(128,30))
        self.relu = nn.LeakyReLU()
        self.sig =nn.Sigmoid()
        
    def forward(self, r, l,  adjacency_r, adjacency_l):
        h_l = self.relu(self.h1(l))
        h_l = F.layer_norm(h_l,[h_l.size()[-1]])
        h_l_i = self.l1(h_l, adjacency_l) + h_l
        h_r = self.relu(self.h1(r))
        h_r = F.layer_norm(h_r,[h_r.size()[-1]])
        h_r_i = self.l1(h_r, adjacency_r) + h_r
        h_r_1,_ = self.attn1(h_l_i,h_r_i,h_l_i)
        h_l_1,_ = self.attn1(h_r_i,h_l_i,h_r_i)
        
        h_l = self.relu(self.h2(h_l_1))
        h_l = F.layer_norm(h_l,[h_l.size()[-1]])
        h_l_i = self.l2(h_l, adjacency_l) + h_l
        h_r = self.relu(self.h2(h_r_1))
        h_r = F.layer_norm(h_r,[h_r.size()[-1]])
        h_r_i = self.l2(h_r, adjacency_r) + h_r
        h_r_1,_ = self.attn2(h_l_i,h_r_i,h_l_i)
        h_l_1,_ = self.attn2(h_r_i,h_l_i,h_r_i)
        
        h_l = self.relu(self.h3(h_l_1))
        h_l = F.layer_norm(h_l,[h_l.size()[-1]])
        h_l_i = self.l3(h_l, adjacency_l) + h_l
        h_r = self.relu(self.h3(h_r_1))
        h_r = F.layer_norm(h_r,[h_r.size()[-1]])
        h_r_i = self.l3(h_r, adjacency_r) + h_r
        h_r_1,_ = self.attn3(h_l_i,h_r_i,h_l_i)
        h_l_1,_ = self.attn3(h_r_i,h_l_i,h_r_i)
        
        h_l = self.relu(self.h4(h_l_1))
        h_l = F.layer_norm(h_l,[h_l.size()[-1]])
        h_l_i = self.l4(h_l, adjacency_l) + h_l
        h_r = self.relu(self.h4(h_r_1))
        h_r = F.layer_norm(h_r,[h_r.size()[-1]])
        h_r_i = self.l4(h_r, adjacency_r) + h_r
        h_r_1,att_2 = self.attn4(h_l_i,h_r_i,h_l_i)
        h_l_1,att_1 = self.attn4(h_r_i,h_l_i,h_r_i)
        
        att = torch.cat([att_1,att_2.transpose(-2,-1)], dim = 1)
        att = self.att_conv(att)
        features = torch.cat([h_r_1,h_l_1], dim = 1)
        features = self.linear_out(features)
#         features = features.view(features.size(0), -1)
#         att = att.view(att.size(0), att.size(-2),att.size(-1))
        return features # self.sig(features),self.sig(att)

In [45]:
class LitMNISTcc(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.criterion = nn.MSELoss() #torch.nn.BCELoss() #nn.MSELoss() pos_weight=self.pos_weight
        self.h_feats_dim = 64 #128#64 #3
        self.att_mlp_Q = nn.Sequential(nn.Linear(self.h_feats_dim, self.h_feats_dim, bias=False),\
                                  nn.LeakyReLU())# get_non_lin(nonlin, leakyrelu_neg_slope),
        self.att_mlp_K = nn.Sequential(nn.Linear(self.h_feats_dim, self.h_feats_dim, bias=False),\
                                  nn.LeakyReLU())#get_non_lin(nonlin, leakyrelu_neg_slope),
        self.att_mlp_V = nn.Linear(self.h_feats_dim, self.h_feats_dim, bias=False)
        self.cross_msgs = True
        self.layer_1 = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, 3))
        self.layer_3 = nn.Linear(128, 3)
        self.epoch_accuracy = []
        self.whole_accuracy = []
        self.epoch_loss = []
        self.whole_loss = []

        self.r=NetR()
        self.l=NetL()
        
        self.lr=0.0001
        self.sig = nn.Sigmoid()
        self.multihead_attn = nn.MultiheadAttention( self.h_feats_dim, 4)
        self.Amean = []
        self.Lmean = []
        self.wholeAmean = []
        self.wholeLmean = []
        
    def compute_cross_attention(self, queries, keys, values, mask, cross_msgs):
        """Compute cross attention.
        x_i attend to y_j:
        a_{i->j} = exp(sim(x_i, y_j)) / sum_j exp(sim(x_i, y_j))
        attention_x = sum_j a_{i->j} y_j
        Args:
          queries: NxD float tensor --> queries
          keys: MxD float tensor --> keys
          values: Mxd
          mask: NxM
        Returns:
          attention_x: Nxd float tensor.
        """
        if not self.cross_msgs:
            return queries * 0.
        mask = mask.to(device)
        a = mask * torch.mm(queries, torch.transpose(keys, 1, 0)) - 1000. * (1. - mask)
        a_x = torch.softmax(a, dim=1)  # i->j, NxM, a_x.sum(dim=1) = torch.ones(N)
        attention_x = torch.mm(a_x, values)  # (N,d)
        return attention_x



    def get_mask(self, ligand_batch_num_nodes, receptor_batch_num_nodes):
        rows = ligand_batch_num_nodes# sum()
        cols = receptor_batch_num_nodes #sum()
        mask = torch.ones(rows, cols)#.to(device)
        return mask
    def cross_attention_layer(self, rec_feat, lig_feat):
#         before = torch.cat((rec_feat, lig_feat), 0)
        
        mask = self.get_mask( lig_feat.shape[0], rec_feat.shape[0])
        cross_ligand = self.compute_cross_attention(self.att_mlp_Q(lig_feat),
                                self.att_mlp_K(rec_feat),
                                self.att_mlp_V(rec_feat),
                                mask,
                                self.cross_msgs)
        cross_receptor = self.compute_cross_attention(self.att_mlp_Q(rec_feat),
                                self.att_mlp_K(lig_feat),
                                self.att_mlp_V(lig_feat),
                                mask.transpose(0,1),
                                self.cross_msgs)
        cross_receptor += rec_feat
        cross_ligand += lig_feat
        
        
#         print(cross_receptor.shape, cross_ligand.shape)
        
#         combine = torch.cat((cross_receptor, cross_ligand),0)#.unsqueeze(0)
#         combine = before + combine
        return cross_receptor, cross_ligand  #combine
    
    def forward(self, re, li, input_edges_rec, input_edges_lig):
        outr = self.r(re.float().to(device), torch.transpose(input_edges_rec, 1,0).to(device))
#         outr= outr.unsqueeze(0)
#         print(outr.shape)
        outr, _ = self.multihead_attn(outr, outr, outr) #.to(device)

        outl = self.l(li.float(), torch.transpose(input_edges_lig, 1,0)) #.to(device)
#         outl= outl.unsqueeze(0)
        outl, _ = self.multihead_attn(outl, outl, outl)
#         print(outr.shape, outl.shape)
        rec, lig = self.cross_attention_layer(outr[0].to(device), outl[0].to(device))
#         out = self.layer_1(out)
#         out = self.sig(out)   #torch.mean(out, dim=0)
        return rec, lig #out


In [46]:
class LitMNIST(LightningModule):
    def __init__(self):
        super().__init__()
        self.h_feats_dim = 64
#         self.criterion = nn.BCEWithLogitsLoss(pos_weight = torch.rand(2)) #FocalLoss() #nn.MSELoss() #torch.nn.BCELoss() #nn.MSELoss() pos_weight=self.pos_weight 
        self.block1 = LitMNISTcc() 
        self.block2 = LitMNISTcc()
        self.block3 = LitMNISTcc()
        self.block4 = LitMNISTcc()
        
        self.spatial = NetRSPA()
        self.out_dim = 44
        self.pos_weight = torch.tensor(1, dtype=torch.float)
        self.layer_1r = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, self.out_dim))
        self.layer_1l = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, self.out_dim))
        self.layer_2r = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, self.out_dim))
        self.layer_2l = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, self.out_dim))
        self.layer_3r = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, self.out_dim))
        self.layer_3l = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, self.out_dim))

        self.layer_2 = nn.Sequential(nn.Linear(94, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                        nn.LeakyReLU(), nn.Linear(256, 1)) ##3
        self.lr=0.0001
        self.sig = nn.Sigmoid()
        self.multihead_attn = nn.MultiheadAttention( self.h_feats_dim, 4)
        self.Amean = []
        self.Lmean = []
        self.wholeAmean = []
        self.wholeLmean = []
        self.criterion = nn.BCEWithLogitsLoss(pos_weight =torch.tensor(10))
    def forward(self, r,l, input_edges_rec, input_edges_lig):
#         print(r.shape)  l*67
        r,l  = self.block1(r[:,3:],l[:,3:], input_edges_rec, input_edges_lig) #.unsqueeze(0)
        sprl = self.spatial(r[:,:3].unsqueeze(0),l[:,:3].unsqueeze(0), input_edges_rec.unsqueeze(0), input_edges_lig.unsqueeze(0))

        com = torch.cat((r,l),0)
        
        concat = torch.cat((com, sprl[0]),1) #l*74
        com = self.layer_2(concat)
#         com = self.sig(com)
        return com
        
    def training_step(self, batch, batch_idx):
        nam, r,l, edge_rec, edge_lig,rl, y = batch
        num_positives = torch.sum(y[0], dim=0)
        num_negatives = len(y[0]) - num_positives
        self.pos_weight  = num_negatives / num_positives

        out = self(r[0], l[0], edge_rec[0], edge_lig[0])
        out = torch.transpose(out, 1,0)
        loss = self.criterion(out.float(), y.float()) #.unsqueeze(0)
        loss= loss.mean()
        binoutput=torch.where(out>=0.5, 1., 0.)
        one_index_out = torch.nonzero(binoutput).flatten().tolist()
        one_index_y = torch.nonzero(y[0]).flatten().tolist()
        accuaracy = len(list(set(one_index_out) & set(one_index_y)))/len(one_index_y)
        result_dict =  {'loss': loss, 'accuracy': accuaracy}
        
        return result_dict #loss
    def test_step(self, batch, batch_idx):
        nam,r,l, edge_rec, edge_lig,rl, y = batch
        num_positives = torch.sum(y[0], dim=0)
        num_negatives = len(y[0]) - num_positives
        self.pos_weight  = num_negatives / num_positives
        out = self(r[0], l[0], edge_rec[0], edge_lig[0])

#         print(x.shape, edge_rec.shape, edge_lig.shape)
#         out = self(x[0], edge_rec[0], edge_lig[0])
        out = torch.transpose(out, 1,0)
        out = torch.mean(out, dim=0)
        binoutput=torch.where(out>=0.5, 1., 0.)
        cpuout = binoutput.flatten().cpu().detach().numpy()
        index_ones = torch.nonzero(torch.from_numpy(cpuout)).flatten()
        reclen = r[0].shape[0]
        allindex=torch.where(index_ones<reclen, index_ones, -(index_ones-reclen))
        print(allindex)
        cpuy = y.flatten().cpu().detach().numpy()
        auc= roc_auc_score(cpuy, cpuout)
        accuracy = accuracy_score(cpuy, cpuout)
        F1_score = f1_score(cpuy, cpuout)
        recall= recall_score(cpuy, cpuout) #binoutput) #
        precision= precision_score(cpuy, cpuout)
        mcc= matthews_corrcoef(cpuy, cpuout)
        aupr = average_precision_score(cpuy, cpuout)
#         confusion = confusion_matrix(cpuy, cpuout, normalize='all')

        return { 'accuracy':accuracy, 'Precision':precision, \
                'Recall':recall, 'F1_score':F1_score, 'auroc':auc,'mcc':mcc, 'aupr': aupr}         
    def training_epoch_end(self, outputs):
        self.Amean = []
        self.Lmean = []
        for i in outputs:
            self.Amean.append(i['accuracy'])
            self.Lmean.append(i['loss'])
        self.wholeAmean.append(torch.mean(torch.tensor(self.Amean)))
        self.wholeLmean.append(torch.mean(torch.tensor(self.Lmean)))
        print(self.wholeLmean[-1])
        self.logger.experiment.add_scalar('loss',torch.mean(torch.tensor(self.Amean)))
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr, betas=(0.1, 0.999))
        return optimizer
    def get_filename(self,ind):
        return np.load('testEpi.npy')[ind]
    
    def test_epoch_end(self, outputs):
        accuracy = []
        loss = []
        Specificity = []
        Precision = []
        Recall =[]
        F1_score = []
        auroc = []
        mcc = []
        aupr = []
        for i in outputs:
            accuracy.append(i['accuracy'])
#             Specificity.append(i['Specificity'])
            Precision.append(i['Precision'])
            Recall.append(i['Recall'])
            F1_score.append(i['F1_score'])
            auroc.append(i['auroc'])
            mcc.append(i['mcc'])
            aupr.append(i['aupr'])
#             adj_acc.append(i['adj accuracy'])
        avg_accuracy= torch.mean(torch.tensor(accuracy).float())
        avg_Precision = torch.mean(torch.tensor(Precision).float())
        avg_Recall = torch.mean(torch.tensor(Recall).float())
        avg_F1_score = torch.mean(torch.tensor(F1_score).float())
        avg_auroc = torch.mean(torch.tensor(auroc).float())
        avg_mcc = torch.mean(torch.tensor(mcc).float())
        avg_aupr = torch.mean(torch.tensor(aupr).float()) 
        
        print('A: ',avg_accuracy,   'P: ',avg_Precision,'R: ',avg_Recall,\
             'F1: ', avg_F1_score, 'AUROC: ',avg_auroc, 'mcc', avg_mcc,'aupr', avg_aupr)    


from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn.functional as F
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.metrics import f1_score,average_precision_score, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, matthews_corrcoef

checkpoint_callback = ModelCheckpoint(monitor="val_loss")
model = LitMNIST()#.to(device)
logger = TensorBoardLogger("tb_logs", name="model")
trainer = Trainer(gpus=[0], max_epochs=100, callbacks=[checkpoint_callback])#,logger=logger)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [47]:
model = LitMNIST().load_from_checkpoint("UnBoundInterfacewt10gcn30feat_500.ckpt")
trainer = Trainer(gpus=[3], max_epochs=1, callbacks=[checkpoint_callback])
trainer.test(model, dataobj) 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0:   0%|                             | 0/130 [00:00<?, ?it/s]tensor([  16,   17,   18,   61,   62,   64,   65,   80,   81,  124,  125,  126,
         127,  184,  185,  186,  187,  188,  189,  190,  191,  192,  208,  209,
         212,  214,  233,  234,  235,  236,  263,  264,  265,  266,  267,  268,
         278,  279,  327,  328,  418,   -8,   -9,  -10,  -11,  -12,  -13,  -14,
         -15,  -16,  -17,  -18,  -19,  -20,  -21,  -34,  -35,  -38,  -39,  -40,
         -41,  -53,  -54,  -55,  -56,  -57,  -58,  -97, -101, -102, -103, -104,
        -105, -106, -107, -109, -111, -112, -113, -114, -115, -116, -117, -118,
        -119, -120, -130, -131, -132, -133, -134, -135, -136, -137, -138, -159,
        -160, -161, -162, -165, -166, -167, -168, -169, -170, -171, -172, -174,
        -175, -176, -178, -180, -195, -196, -197, -198, -199])
Testing DataLoader 0:   1%|▏                    | 1/130 [00:03<07:53,  3.67s/it]tensor([   7,    8,   35,   36,   37,   38,   39,   40,  

Testing DataLoader 0:  11%|██▏                 | 14/130 [00:05<00:13,  8.41it/s]tensor([  41,   42,   43,   45,   59,   60,   61,   62,   63,   64,  113,  114,
         115,  116,  117,  118,  119,  120,  121,  122,  123,  124,  132,  133,
         134,  135,  136,  137,  138,  139,  140,  142,  162,  163,  164,  165,
         169,  176,  177,  178,  179,  180,  181,  182,  183,  184,  185,  205,
         209,  211,  212,  214,  220,  -16,  -17,  -18,  -19,  -39,  -40,  -41,
        -144, -145, -146, -147, -188, -189, -190, -191, -195, -196, -197, -198,
        -199, -206, -207, -208, -209, -219, -220, -221, -222, -231, -232, -233,
        -267, -269, -324, -325, -326, -327, -337, -338, -339, -361, -362, -363,
        -364, -365, -367, -368, -369])
Testing DataLoader 0:  12%|██▎                 | 15/130 [00:05<00:13,  8.41it/s]tensor([ 164,  165,  166,  167,  168,  231,  232,  233,  234,  235,  236,  237,
         270,  271,  272,  273,  274,   -4,   -5,   -6,   -7,   -8,  -10,  -11,
 

Testing DataLoader 0:  22%|████▍               | 29/130 [00:05<00:05, 18.52it/s]tensor([   0,    1,    2,   23,   24,   25,   26,   28,   70,   71,   72,   73,
          74,   75,   76,  137,  139,  154,  162,  163,  165,  166,  167,  168,
         174,  175,  184,  185,  186,  321,  322,  323,  324,  339,  340,  341,
         342,  343,  365,  366,  367,  368,  369,  383,  384,  385,   -4,   -5,
          -6,   -7,   -8,   -9,  -10,  -15,  -16,  -17,  -18,  -19,  -20,  -22,
         -33,  -34,  -35,  -36,  -41,  -42,  -43,  -44,  -45,  -46,  -47,  -48,
         -49,  -50,  -51,  -60,  -61,  -62,  -63,  -64,  -67,  -86,  -91,  -92,
         -93, -105, -106, -107, -108, -109, -110])
Testing DataLoader 0:  23%|████▌               | 30/130 [00:05<00:05, 18.52it/s]tensor([   5,    6,    7,   19,   20,   22,   44,   71,   72,   73,   74,   76,
          78,   95,   96,   97,   99,  100,  207,  208,  209,  211,  212,  213,
         214,  215,  216,  225,  226,  227,  228,  230,  231,  232,  

Testing DataLoader 0:  32%|██████▎             | 41/130 [00:06<00:05, 14.96it/s]tensor([ 129,  150,  151,  152,  153,  154,  155,  156,  157,  158,  159,  161,
         164,  165,  167,  188,  189,  190,  191,  192,  193,  195,  204,  205,
         206,  207,  208,  209,  210,  223,  224,  225,  226,  227,  228,  229,
         230,  231,  232,  233,  270,  271,  272,  273,  274,  275,  276,  289,
         290,  291,  293,  294,  295,  319,  320,  321,  322,  323,  324,  325,
         354,  355,  356,  377,  378,  379,  380,  381,  382,  383,  384,  385,
         386,  387,  -20,  -21,  -22,  -44,  -45,  -58,  -59,  -60,  -61,  -62,
         -63,  -95,  -96,  -97,  -98,  -99, -109, -110, -111, -112, -113, -114,
        -115, -146, -147, -148, -149, -151, -152, -154, -155, -156, -157, -158,
        -159, -160, -161, -162, -163, -184, -185, -186, -187, -188, -198, -199,
        -200, -208, -214])
Testing DataLoader 0:  32%|██████▍             | 42/130 [00:06<00:05, 15.99it/s]tensor([119, 

...

Testing DataLoader 0:  67%|█████████████▍      | 87/130 [00:09<00:02, 20.10it/s]tensor([  52,   53,   54,   55,   68,   71,   98,  201,  212,  213,  214,  222,
         223,  224,  232,  233,  234,  235,  236,  237,  238,  239,  240,  241,
         242,  247,  248,  249,  250,  251,  252,  253,  254,  255,  256,  257,
         264,  265,  266,  267,  268,  272,  273,  274,  280,  281,  282,  283,
         284,  285,  303,  304,  305,  306,  307,  308,  309,  323,  326,  327,
         365,  366,  367,  368,  369,  370,  -11,  -12,  -13,  -27,  -28,  -29,
         -30,  -31,  -62,  -63,  -94,  -95,  -96,  -97,  -98,  -99, -100, -101,
        -102, -103, -104, -141, -142, -143, -144, -159, -160, -161, -162, -163,
        -164, -175, -176, -177, -178, -183, -185, -186, -187, -188])
Testing DataLoader 0:  68%|█████████████▌      | 88/130 [00:09<00:02, 20.10it/s]tensor([  33,   34,   35,   39,   40,   45,   48,   53,   60,   63,   64,   66,
          79,   92,   93,   94,   95,  125,  126,  

Testing DataLoader 0:  75%|██████████████▉     | 97/130 [00:10<00:01, 17.99it/s]tensor([ 126,  127,  133,  134,  135,  136,  137,  140,  157,  158,  159,  160,
         161,  162,  163,  164,  166,  167,  168,  169,  170,  176,  177,  178,
         179,  181,  183,  318,  340,  341,  366,  367,  380,   -8,  -20,  -21,
         -22,  -25,  -26,  -36,  -37,  -38,  -39,  -40,  -45,  -46,  -48,  -49,
         -50,  -51,  -52,  -55,  -57,  -60,  -61,  -64,  -75,  -77,  -79,  -80,
         -81,  -82,  -83,  -84,  -85,  -91,  -92,  -93,  -94,  -95, -101, -102,
        -103, -104, -105, -106, -107, -108, -109, -110])
Testing DataLoader 0:  75%|███████████████     | 98/130 [00:10<00:01, 19.35it/s]tensor([  34,   49,   50,   51,   53,   54,   55,   56,   57,  124,  154,  157,
         158,  159,  160,  188,  189,  190,  191,  192,  209,  210,  211,  212,
         213,  214,  215,  216,  217,  218,  219,  220,  221,  233,  235,  236,
         237,  238,  239,  240,  241,  242,  243,  253,  274,  

Testing DataLoader 0:  82%|███████████████▋   | 107/130 [00:10<00:01, 20.29it/s]tensor([ 142,  144,  146,  147,  148,  149,  150,  177,  181,  182,  183,  185,
         187,  188,  189,  190,  191,  194,  195,  196,  197,  198,  199,  200,
         202,  206,  207,  208,  229,  230,  231,  232,  233,  234,  256,  257,
         258,  259,  262,  264,  268,  269,  270,  271,  272,  273,  274,  275,
         276,  277,    0,   -2,   -4,   -5,   -6,   -7,  -14,  -15,  -16,  -19,
         -20,  -22,  -24,  -25,  -27,  -36,  -37,  -39,  -40,  -41,  -63,  -64,
         -65,  -66,  -87,  -88,  -89,  -91,  -92,  -99, -100, -111, -113, -115,
        -117, -119, -120, -121, -122, -123, -124, -125, -126, -127, -128, -129,
        -130, -131, -134, -135, -136, -137, -138, -139, -160, -161, -162, -163,
        -164, -165, -168, -171, -173, -175, -176, -177, -178, -189])
Testing DataLoader 0:  83%|███████████████▊   | 108/130 [00:10<00:01, 20.29it/s]tensor([  18,   35,   36,   37,   38,   41,   42,  

Testing DataLoader 0:  92%|█████████████████▌ | 120/130 [00:11<00:00, 14.83it/s]tensor([ 156,  157,  159,  160,  161,  162,  163,  164,  227,  228,  229,  236,
         237,  238,  239,  240,  241,  260,  261,  262,  263,  264,  265,  266,
         267,  268,  269,  270,  271,  272,  273,  274,  275,  276,  277,  278,
         279,  367,  368,  369,  370,   -5,   -6,   -8,   -9,  -10,  -11,  -12,
         -13,  -14,  -15,  -19,  -20,  -36,  -37,  -38,  -39,  -40,  -54,  -66,
         -67,  -68,  -69,  -70,  -72,  -73,  -74,  -75,  -76,  -97,  -98,  -99,
        -120, -121, -203])
Testing DataLoader 0:  93%|█████████████████▋ | 121/130 [00:11<00:00, 14.83it/s]tensor([  60,   61,   62,   63,   64,   65,  144,  145,  146,  149,  150,  151,
         152,  153,  154,  155,  156,  157,  158,  159,  160,  161,  162,  171,
         203,  204,  228,  229,  230,  231,  232,  270,  271,  294,  295,  296,
         316,  317,  338,  344,  346,  347,  363,  364,  365,  376,  377,  396,
         397,

[{}]

In [ ]:
# EpipredUnBoundInterfacewt10gcn30feat_150part_"+str(k)+".ckpt"
model = LitMNIST().load_from_checkpoint("EpipredUnBoundInterfacewt10gcn30feat_150part_4.ckpt")
trainer = Trainer(gpus=[3], max_epochs=1, callbacks=[checkpoint_callback])
trainer.test(model, dataobj) 

In [15]:
trainer.fit(model, dataobj)
trainer.save_checkpoint("UnBoundInterfacewt10gcn30feat_100.ckpt") #Interfacegcn_100.ckpt
# trainer.save_checkpoint("InterfaceProbabilityPredictionBhSTNGCN44_300_1blockBCEnoposwtnoorm.ckpt")
trainer.test(model, dataobj) 

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

   | Name           | Type               | Params
-------------------------------------------------------
0  | block1         | LitMNISTcc         | 2.8 M 
1  | block2         | LitMNISTcc         | 2.8 M 
2  | block3         | LitMNISTcc         | 2.8 M 
3  | block4         | LitMNISTcc         | 2.8 M 
4  | spatial        | NetRSPA            | 7.8 M 
5  | layer_1r       | Sequential         | 52.7 K
6  | layer_1l       | Sequential         | 52.7 K
7  | layer_2r       | Sequential         | 52.7 K
8  | layer_2l       | Sequential         | 52.7 K
9  | layer_3r       | Sequential         | 52.7 K
10 | layer_3l       | Sequential         | 52.7 K

Epoch 1:   0%|          | 1/1547 [00:00<04:34,  5.64it/s, loss=1.39, v_num=1115]

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:378: UserWarning: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
  warning_cache.warn(m)


Epoch 99: 100%|█████| 1547/1547 [01:27<00:00, 17.75it/s, loss=0.524, v_num=1115]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Testing DataLoader 0:  49%|██████████▏          | 20/41 [00:00<00:00, 25.25it/s][0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Testing DataLoader 0:  63%|█████████████▎       | 26/41 [00:01<00:00, 26.89it/s][0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Testing DataLoader 0:  80%|████████████████▉    | 33/41 [00:01<00:00, 28.11it/s][1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Testing DataLoader 0:  93%|███████████████████▍ | 38/41 [00:01<00:00, 28.28it/s][1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
Testing DataLoader 0:  95%|███████████████████▉ | 39/41 [00:01<00:00, 26.84it/s][1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1

[{}]

In [27]:
model = LitMNIST().load_from_checkpoint("UnBoundInterfacewt10gcn30feat_100.ckpt")
trainer = Trainer(gpus=[3], max_epochs=50, callbacks=[checkpoint_callback])
trainer.fit(model, dataobj)
trainer.save_checkpoint("UnBoundInterfacewt10gcn30feat_150.ckpt")
trainer.test(model, dataobj)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

   | Name           | Type               | Params
-------------------------------------------------------
0  | block1         | LitMNISTcc         | 2.8 M 
1  | block2         | LitMNISTcc         | 2.8 M 
2  | block3         | LitMNISTcc         | 2.8 M 
3  | block4         | LitMNISTcc         | 2.8 M 
4  | spatial        | NetRSPA            | 7.8 M 
5  | layer_1r       | Sequential         | 52.7 K
6  | layer_1l       | Sequential         | 52.7 K
7  | layer_2r       | Sequential         | 52.7 K
8  | layer_2l       | Sequential         | 52.7 K
9  | layer_3r       | Sequential         | 52.7 K
10 | layer_3l       | Sequential         | 52.7 K
11 | layer_2        | Sequential         | 45.4 K
12 | sig            | Sigmoid            | 0     
13 | multihead_attn | MultiheadAttention | 16.6 K


Epoch 49: 100%|█████| 1547/1547 [01:30<00:00, 17.06it/s, loss=0.421, v_num=1119]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0:   0%|                              | 0/41 [00:00<?, ?it/s][0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
Testing DataLoader 0:  15%|███▏                  | 6/41 [00:00<00:01, 17.77it/s][1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 

...

Testing DataLoader 0:  49%|██████████▏          | 20/41 [00:01<00:01, 20.88it/s][1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Testing DataLoader 0:  63%|█████████████▎       | 26/41 [00:01<00:00, 22.32it/s][0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1

Testing DataLoader 0:  80%|████████████████▉    | 33/41 [00:01<00:00, 23.97it/s][1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0

Testing DataLoader 0:  93%|███████████████████▍ | 38/41 [00:01<00:00, 22.52it/s][1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
Testing DataLoader 0:  95%|███████████████████▉ | 39/41 [00:01<00:00, 22.52it/s][1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0

[{}]